In [7]:
import pandas as pd
import os
import chardet
import re
import logging

def process_txt_file(file):
     #检测文件编码
     with open(file, 'rb') as f:
         detacted = chardet.detect(f.read(4096))
         encoding = detacted['encoding']
         print(f'正在处理{file} | 文件编码{encoding}')
         
     #按行读取文件
     with open(file, 'r', encoding = encoding) as f:
         lines = f.readlines()
         
     #清洗文件中的特殊字符，分列，制表
     table0 = [line.split('|') for line in lines[1:]]
     table = []
     for line in table0:
         table.append([cell.strip() for cell in line[0:17]])
     df = pd.DataFrame(data = table[1:], columns= table[0])
     col = str(df.columns)
     print(f'行名: {col}')
     #AKSHARE的列名对应关系字典
     options_column_mapping ={
        "合约代码" : "option_name",
        "交易日期" : "trade_date",
        "今开盘" : "open",
        "最高价" : "high",
        "最低价" : "low",
        "今收盘" : "close",
        "昨结算" : "pre_settle",
        "今结算" : "settle",
        "DELTA" : "delta",
        "成交量(手)" : "volume",
        "持仓量" : "open_interest",
        "成交额(万元)" : "trnover",
        "行权量" : "exercise_vol"
    }

     #保留必要的列
     columns_to_keep = list(options_column_mapping.keys())
     df = df[columns_to_keep]

     #更改列名
     df = df.rename(columns = options_column_mapping)

     #将YYYY-MM-DD格式日期改为YYYYMMDD格式
     df['trade_date'] =pd.to_datetime(df['trade_date']).dt.strftime('%Y%m%d')
     print('文件处理完毕')
     return df

def combine_txt_files(folder_path):
    """遍历指定目录下的所有 TXT 文件，并合并它们。"""
    file_list = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

    all_df = []
    
    for filename in file_list:
        filepath = os.path.join(folder_path, filename)
        df = process_txt_file(filepath) #传递column_mapping参数

        if df is None:
            logging.error(f"处理文件 {filepath} 失败，停止合并。")
            return None

        all_df.append(df)
    
    if not all_df: #如果没有任何文件被成功读取，则返回None
        logging.warning(f"没有成功读取任何文件。")
        return None

    combined_df = pd.concat(all_df, ignore_index=True)
    logging.info(f"文件夹 {folder_path} 处理完成，共合并 {len(combined_df)} 行数据。")
    return combined_df

folder_path = r"D:\JupyterLabFiles\Futures\Futures_exchange_data\CZCE\Options"  # 替换为实际的文件夹路径

combined_df = combine_txt_files(folder_path) #传递column_mapping参数

if combined_df is not None:
    print(combined_df.head())
    combined_df.to_csv("combined_Options.csv", index=False, encoding='gbk')
    logging.info(f"合并后的数据已保存到 combined_txt1.csv。")
else:
    logging.error("处理过程中出现错误。")

正在处理D:\JupyterLabFiles\Futures\Futures_exchange_data\CZCE\Options\ALLOPTIONS2017.txt | 文件编码utf-8
行名: Index(['交易日期', '合约代码', '昨结算', '今开盘', '最高价', '最低价', '今收盘', '今结算', '涨跌1', '涨跌2',
       '成交量(手)', '持仓量', '增减量', '成交额(万元)', 'DELTA', '隐含波动率', '行权量'],
      dtype='object')
文件处理完毕
正在处理D:\JupyterLabFiles\Futures\Futures_exchange_data\CZCE\Options\ALLOPTIONS2018.txt | 文件编码utf-8
行名: Index(['交易日期', '合约代码', '昨结算', '今开盘', '最高价', '最低价', '今收盘', '今结算', '涨跌1', '涨跌2',
       '成交量(手)', '持仓量', '增减量', '成交额(万元)', 'DELTA', '隐含波动率', '行权量'],
      dtype='object')
文件处理完毕
正在处理D:\JupyterLabFiles\Futures\Futures_exchange_data\CZCE\Options\ALLOPTIONS2019.txt | 文件编码utf-8
行名: Index(['交易日期', '合约代码', '昨结算', '今开盘', '最高价', '最低价', '今收盘', '今结算', '涨跌1', '涨跌2',
       '成交量(手)', '持仓量', '增减量', '成交额(万元)', 'DELTA', '隐含波动率', '行权量'],
      dtype='object')
文件处理完毕
正在处理D:\JupyterLabFiles\Futures\Futures_exchange_data\CZCE\Options\ALLOPTIONS2020.txt | 文件编码utf-8
行名: Index(['交易日期', '合约代码', '昨结算', '今开盘', '最高价', '最低价', '今收盘', '今结算', '涨跌